In [4]:
from random import shuffle
import glob, h5py, pickle, cv2, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
sub = 'projected'

trainpath = '/mnt/HDD04/Projection_data/OUTPUTS/Cascade_AWR2243/' + sub + '_microDoppler/*.png'
filename = 'datasets/' + sub + '.hdf5'
print('File to be created: ' + filename)

File to be created: datasets/projected.hdf5


In [2]:
train_files2 = sorted(glob.glob(trainpath))
print(len(train_files2))
train_images = []
cnt = 1
train_labels = []
for i in range(len(train_files2)):
    fname = train_files2[i].split('/')[-1]
    if '\\' in fname:
        fname = fname.split('\\')[-1]
    class_idx = [m.end() for m in re.finditer('class', fname)]
    underscore_idx = [m.start() for m in re.finditer('_', fname)]
    classes = np.array([int(c) for c in fname[class_idx[0]: underscore_idx[1]].split('-')])

    angles = fname.split('_')[2][3:]
    plusses = angles.rfind('+')
    minuses = angles.rfind('-')

    if plusses == -1 and minuses == -1:
        all_ang_ids = int(angles)
        angs = all_ang_ids
    elif plusses == -1 and minuses != -1:
        all_ang_ids = sorted([minuses, len(angles)+1])
    elif plusses != -1 and minuses == -1:
        all_ang_ids = sorted([plusses, len(angles)+1])
    else:
        all_ang_ids = sorted([plusses, minuses, len(angles)+1])

    if isinstance(all_ang_ids, list):
        if all_ang_ids[0] == 0:
            angs = np.zeros((len(all_ang_ids) - 1,), dtype=int)
            for i in range(len(all_ang_ids) - 1):
                angs[i] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
        else:
            angs = np.zeros((len(all_ang_ids),), dtype=int)
            angs[0] = int(angles[:all_ang_ids[0]])
            for i in range(len(all_ang_ids) - 1):
                angs[i+1] = int(angles[all_ang_ids[i]:all_ang_ids[i+1]]) # .astype(int)
    proj_ang = int(fname[fname.find('proj')+4: -4])
    clas = classes[np.argmax([1 if proj_ang == a else 0 for a in angs])]
        
    if clas == 13:
        clas = 8
    elif clas == 14:
        clas = 9
        
    train_labels.append(clas-1)
    img = cv2.imread(train_files2[i])
    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    clear_output(wait=True)
    print('Loading image '+str(cnt)+'/'+str(len(train_files2))+ ' ' + fname)
    train_images.append(img)  
    cnt += 1
print(np.unique(train_labels))
train_labels = to_categorical(np.array(train_labels))
train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
print(train_images.shape)
print(train_labels.shape)

Loading image 48/48 pCemreRony_class6-1_ang0+45_iter4_weight2_proj45.png
[0 1 5]
(48, 128, 128, 3)
(48, 6)


In [5]:
f = h5py.File(filename, mode='w')
f.create_dataset("train_img", train_images.shape, np.uint8)
# f.create_dataset("test_img", test_images.shape, np.uint8)  
f.create_dataset("train_labels", train_labels.shape, np.uint8)
# f.create_dataset("test_labels", test_labels.shape, np.uint8)

f["train_img"][...] = train_images
# f["test_img"][...] = test_images
f["train_labels"][...] = train_labels
# f["test_labels"][...] = test_labels
f.close()
print(filename+' created.')

datasets/projected.hdf5 created.
